In [1]:
import numpy as np

## Part 1

    1. ratings.dat encapsulating class

In [2]:
class RatingsData(object):
    def __init__(self, raw_data_path):
        with open(raw_data_path, 'r') as f:
            content = f.read()

        self._ratings_data = {}

        for line in content.split('\n'):
            if not line:
                continue
                
            values = line.split('::')
            user_ratings = self._ratings_data.get(values[0], {})
            user_ratings[values[1]] = float(values[2])
            self._ratings_data[values[0]] = user_ratings

    def as_matrix(self, movies):

        sorted_users = sorted(self._ratings_data.keys())

        matrix = []

        for user_id in sorted_users:
            row = []
            for movie in movies:
                row.append(self._ratings_data[user_id].get(movie, np.NaN))
            matrix.append(row)

        return np.array(matrix)

    2. movies.dat encapsulating class

In [3]:
class MoviesData(object):

    def __init__(self, raw_data_path):
        with open(raw_data_path, 'r') as f:
            content = f.read()

        self._movies_data = {}

        for line in content.split('\n'):
            if not line:
                continue
                
            values = line.split('::')
            self._movies_data[values[0]] = {
                'name': values[1],
                'genres': values[2].split('|')
            }

    def get_available_movies(self):
        return sorted(self._movies_data.keys())

    3. Train-Test split function

In [4]:
def split_train_test(ratings_matrix, ratio=0.8):

    train = []
    test = []

    for i, user_ratings in enumerate(ratings_matrix):
        known_ratings = np.argwhere(~np.isnan(user_ratings)).flatten()
        train_ratings = np.random.choice(known_ratings, int(len(known_ratings) * ratio), replace=False)
        test_ratings = np.setdiff1d(known_ratings, train_ratings)

        row = user_ratings.copy()
        mask = np.ones(row.shape, dtype=bool)
        mask[train_ratings] = False
        row[mask] = np.NaN
        train.append(row)
  
        row = user_ratings.copy()
        mask = np.ones(row.shape, dtype=bool)
        mask[test_ratings] = False
        row[mask] = np.NaN
        test.append(row)

    return np.vstack(train), np.vstack(test)

## Part 2

1. Required **hyper**parameters:  
  * $k$ - Dimensionality factor
  * $\lambda_u$ - Users rows regularization factor
  * $\lambda_v$ - Movies rows regularization factor
  * $\lambda_{b_u}$ - Users rows bias regularization factor
  * $\lambda_{b_v}$ - Movies rows bias regularization factor

In [5]:
class Hyperparameters(object):

    def __init__(self, dim, reg_users=1e-3, reg_items=1e-3, reg_bias_users=1e-3, reg_bias_items=1e-3):
        self.k = dim
        self.lambda_u = reg_users
        self.lambda_v = reg_items
        self.lambda_bu = reg_bias_users
        self.lambda_bv = reg_bias_items

    2. Model parameters encapsulating class

In [6]:
class MFModel(object):

    def __init__(self, hyperparams, num_of_users, num_of_movies):
        self.hyper = hyperparams
        self.U = np.random.randn(hyperparams.k, num_of_users)
        self.V = np.random.randn(hyperparams.k, num_of_movies)
        self.Ub = np.random.randn(num_of_users, 1)
        self.Vb = np.random.randn(num_of_movies, 1)

## Part 3

    1.
$$ \frac{\partial E}{\partial U_m} = 
\sum_{n \in I_m} -\left ( r_{m,n} - \left (U_m^TV_n + b_n + b_m \right ) \right ) V_n + \lambda_UU_m$$

$$ \frac{\partial E}{\partial V_n} = 
\sum_{m \in I_n} -\left ( r_{m,n} - \left (U_m^TV_n + b_n + b_m \right ) \right ) U_m + \lambda_VV_n$$

$$ \frac{\partial E}{\partial b_m} = 
\sum_{n \in I_m} -\left ( r_{m,n} - \left (U_m^TV_n + b_n + b_m \right ) \right ) + \lambda_{b_u}b_m$$

$$ \frac{\partial E}{\partial b_n} = 
\sum_{m \in I_m} -\left ( r_{m,n} - \left (U_m^TV_n + b_n + b_m \right ) \right ) + \lambda_{b_v}b_n$$

    2. Stochastic Gradient Descent
       (a) Update equations

In [7]:
def derive_by_um(R, U, V, Ub, Vb, lambda_u, m):
    predict = np.vectorize(lambda n: np.dot(U.T[m], V[:, n]) + Vb[n] + Ub[m])
    ns = np.argwhere(~np.isnan(R[m])).reshape(-1)
    error = (R[m, ns] - predict(ns)).reshape(-1, 1) * -V[:, ns].T

    return error.sum(axis=0) + lambda_u * U[:, m]


def derive_by_vn(R, U, V, Ub, Vb, lambda_v, n):
    predict = np.vectorize(lambda m: np.dot(U.T[m], V[:, n]) + Vb[n] + Ub[m])
    ms = np.argwhere(~np.isnan(R[:, n])).reshape(-1)
    error = (R[ms, n] - predict(ms)).reshape(-1, 1) * -U[:, ms].T

    return error.sum(axis=0).reshape(-1) + lambda_v * V[:, n]


def derive_by_bm(R, U, V, Ub, Vb, lambda_bu, m):
    predict = np.vectorize(lambda n: np.dot(U.T[m], V[:, n]) + Vb[n] + Ub[m])
    ns = np.argwhere(~np.isnan(R[m]))
    error = -(R[m, ns] - predict(ns))

    return error.sum() + lambda_bu * Ub[m]


def derive_by_bn(R, U, V, Ub, Vb, lambda_bv, n):
    predict = np.vectorize(lambda m: np.dot(U.T[m], V[:, n]) + Vb[n] + Ub[m])
    ms = np.argwhere(~np.isnan(R[:, n]))
    error = -(R[ms, n] - predict(ms))

    return error.sum() + lambda_bv * Vb[n]

        (b) SGD parameters encapsulating class

In [23]:
class SGDParameters(object):
    
    def __init__(self, step_size=1e-4, epocs=50):
        self.alpha = step_size
        self.epocs = epocs

        (c) LearnModelFromDataUsingSGD implementation

In [9]:
def LearnModelFromDataUsingSGD(dataset, model_params, algorithm_params):
    U, V, Ub, Vb = model_params.U, model_params.V, model_params.Ub, model_params.Vb

    for epoc in range(algorithm_params.epocs):
        print 'Epoch: ', epoc
        print 'RMSE: ', RMSE(U, V, Ub, Vb, dataset)

        for m, n in np.argwhere(~np.isnan(dataset)):

            U[:, m] -= algorithm_params.alpha * derive_by_um(dataset, U, V, Ub, Vb, model_params.hyper.lambda_u, m)
            V[:, n] -= algorithm_params.alpha * derive_by_vn(dataset, U, V, Ub, Vb, model_params.hyper.lambda_v, n)
            Ub[m] -= algorithm_params.alpha * derive_by_bm(dataset, U, V, Ub, Vb, model_params.hyper.lambda_bu, m)
            Vb[n] -= algorithm_params.alpha * derive_by_bn(dataset, U, V, Ub, Vb, model_params.hyper.lambda_bv, n)

    return U, V, Ub, Vb

## Test

In [20]:
movies = MoviesData('movies.dat')
ratings = RatingsData('ratings.dat')

full_dataset = ratings.as_matrix(movies.get_available_movies())

In [ ]:
reduced_dataset = full_dataset[:1000, :500]
mask = ~np.isnan(reduced_dataset)
reduced_dataset = reduced_dataset[mask.sum(axis=1) > 0]
mask = ~np.isnan(reduced_dataset)
reduced_dataset = reduced_dataset[:, mask.sum(axis=0) > 0]

In [24]:
model_params = MFModel(Hyperparameters(100), *full_dataset.shape)
algorithm_params = SGDParameters(epocs=50)

train_set, test_set = split_train_test(full_dataset)

In [16]:
def RMSE(U, V, Ub, Vb, dataset):
    pred = np.dot(U.T, V) + Ub.reshape(-1, 1) + Vb.reshape(-1, 1).T
    mask = ~np.isnan(dataset)
    return np.sqrt(np.power(pred[mask] - dataset[mask], 2).sum() / mask.sum())

In [25]:
params = LearnModelFromDataUsingSGD(train_set, model_params, algorithm_params)

Epoch:  0
RMSE:  10.765480402483208
Epoch:  1
RMSE:  1.159431987821029
Epoch:  2
RMSE:  0.8842700740014439
Epoch:  3
RMSE:  0.811985251105999
Epoch:  4
RMSE:  0.7781430688356192
Epoch:  5
RMSE:  0.7573423744701497
Epoch:  6
RMSE:  0.7423098307474274
Epoch:  7
RMSE:  0.7303459008883584
Epoch:  8
RMSE:  0.7202509059807491


KeyboardInterrupt: 